In [ ]:
# !pip install --upgrade openai
# !pip install openai==0.28

In [ ]:
!python run_api_gen_data.py --prompt_name gen_scene --character CassiusDio

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="gemma",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    # {"role": "user", "content": "Who won the world series in 2020?"},
    # {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    # {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

In [ ]:
# Ollama
# client = OpenAI(
# base_url = 'http://localhost:11434/v1',
# api_key='ollama', # required, but unused
# )
# response = client.chat.completions.create(
#     model="gemma",
#     messages=[
#         {"role": "system", "content": sys_prompt_content},
#         {"role": "user", "content": input},
#         ],
#     max_tokens=max_tokens,
#     temperature=temperature,
#     top_p=top_p,
#     frequency_penalty=frequency_penalty,
#     presence_penalty=presence_penalty,
#     n=n,
#     stop=stop,
# )

In [ ]:
from groq import Groq

client = Groq(
    api_key='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',
)

completion = client.chat.completions.create(
    model="mixtral-8x7b-32768",
    messages=[
        {
            "role": "user",
            "content": ""
        }
    ],
    temperature=0.5,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


In [ ]:
# !python parser/parse_data_scene.py result/2023-10-08/gen_scene/gpt-3.5-turbo-temp-0.7-char-CassiusDio.jsonl

In [ ]:
# !python run_api_gen_data.py --prompt_name gen_dialogue --character CassiusDio --data_path processed/2023-10-08/

In [ ]:
# !python parser/parse_data_dialogue.py result/2023-10-08/gen_dialogue/gpt-3.5-turbo-temp-0.7-char-CassiusDio.jsonl

In [ ]:
# python run_api_gen_data.py --prompt_name gen_hallucination --character CassiusDio --data_path processed/2023-10-08/

In [ ]:
# python parser/parse_data_hallucination.py result/2023-10-08/gen_hallucination/gpt-3.5-turbo-temp-0.7-char-CassiusDio.jsonl

In [ ]:
python parser/convert_prompt_data.py processed/2023-10-08/generated_agent_dialogue_CassiusDio.json

In [ ]:
# conda install pytorch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 pytorch-cuda=12.1 -c pytorch -c nvidia
# conda install cuda -c nvidia
# pip3 install "fschat[model_worker,webui]"
# git clone https://github.com/lm-sys/FastChat.git
# cd FastChat
# pip3 install -e ".[train]"

# cd /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM
# conda activate vllmenv



# ENVARS ADDED **ONLY FOR READABILITY**
NVIDIA_CUDA_PPA=https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/
NVIDIA_CUDA_PREFERENCES=https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
NVIDIA_CUDA_PUBKEY=https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/7fa2af80.pub

# Add NVIDIA Developers 3rd-Party PPA
sudo wget ${NVIDIA_CUDA_PREFERENCES} -O /etc/apt/preferences.d/nvidia-cuda
sudo apt-key adv --fetch-keys ${NVIDIA_CUDA_PUBKEY}
echo "deb ${NVIDIA_CUDA_PPA} /" | sudo tee /etc/apt/sources.list.d/nvidia-cuda.list

# Install development tools
sudo apt update
sudo apt install -y cuda

In [20]:
import json

train_json = json.load(open('processed\\2023-10-08\\prompted\\prompted_agent_dialogue_CassiusDio.jsonl', "r"))

In [ ]:
# cd C:\Users\rs659\Desktop\Pliny\Character-LLM


# Scene Extraction
# python run_api_gen_data.py --prompt_name gen_scene --character Beethoven --debug 1
# python parser/parse_data_scene.py result/2023-10-08/gen_scene/gpt-3.5-turbo-temp-0.7-char-Beethoven.jsonl

# Experience Completion
# python run_api_gen_data.py --prompt_name gen_dialogue --character Beethoven --data_path processed/2023-10-08/ --debug 1
# python parser/parse_data_dialogue.py result/2023-10-08/gen_dialogue/gpt-3.5-turbo-temp-0.7-char-Beethoven.jsonl

# Protective Scene
# python run_api_gen_data.py --prompt_name gen_hallucination --character Beethoven --data_path processed/2023-10-08/ --debug 1
# python parser/parse_data_hallucination.py result/2023-10-08/gen_hallucination/gpt-3.5-turbo-temp-0.7-char-Beethoven.jsonl


# Convert to Training Format
# python parser/convert_prompt_data.py processed/2023-10-08/generated_agent_dialogue_Beethoven.json


In [ ]:
# cd FastChat
# export CHARACTER=CassiusDio


torchrun --nproc_per_node=1 --master_port=20031 fastchat/train/train_mem.py --model_name_or_path unsloth/gemma-2b --data_path /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl  --output_dir /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/ckpt/Beethoven_7b --num_train_epochs 10

torchrun --nproc_per_node=1 --master_port=20031 fastchat/train/train_mem.py --model_name_or_path unsloth/gemma-2b --data_path ../processed/2023-10-08/prompted/prompted_agent_dialogue_CassiusDio.jsonl  --output_dir ../ckpt/${CHARACTER}_7b --num_train_epochs 10

torchrun --nproc_per_node=1 --master_port=20031 fastchat/train/train_mem.py --model_name_or_path winglian/Llama-2-3b-hf --data_path /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl   --already_preprocess True --bf16 False --output_dir /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/ckpt/Beethoven_7b --num_train_epochs 1 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --gradient_accumulation_steps 4 --evaluation_strategy epoch --save_strategy epoch --save_total_limit 10 --learning_rate 2e-5 --weight_decay 0.1 --warmup_ratio 0.04 --lr_scheduler_type cosine --logging_steps 1 --fsdp 'full_shard auto_wrap' --fsdp_transformer_layer_cls_to_wrap LlamaDecoderLayer --tf32 False --model_max_length 512 --gradient_checkpointing True

# Working
torchrun --nproc_per_node=1 --master_port=20031 fastchat/train/train_mem.py --model_name_or_path winglian/Llama-2-3b-hf --data_path /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/processed/2023-10-08/prompted/prompted_agent_dialogue_Beethoven.jsonl   --already_preprocess True --bf16 True --output_dir /mnt/c/Users/rs659/Desktop/Pliny/Character-LLM/ckpt/Beethoven_7b --num_train_epochs 1 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --gradient_accumulation_steps 4 --evaluation_strategy epoch --save_strategy epoch --save_total_limit 10 --learning_rate 2e-5 --weight_decay 0.1 --warmup_ratio 0.04 --lr_scheduler_type cosine --logging_steps 1 --fsdp 'full_shard auto_wrap' --fsdp_transformer_layer_cls_to_wrap LlamaDecoderLayer --tf32 True --model_max_length 2048 --gradient_checkpointing True






!torchrun --nproc_per_node=1 --master_port=20031 fastchat/train/train_mem.py --model_name_or_path meta-llama/Llama-2-7b-hf --data_path /path/to/prompted_agent_dialogue_$CHARACTER.json --already_preprocess True --bf16 True --output_dir /path/to/ckpt/${CHARACTER}_7b --num_train_epochs 10 --per_device_train_batch_size 1 --per_device_eval_batch_size 16 --gradient_accumulation_steps 4 --evaluation_strategy epoch --save_strategy epoch --save_total_limit 10 --learning_rate 2e-5 --weight_decay 0.1 --warmup_ratio 0.04 --lr_scheduler_type cosine --logging_steps 1 --fsdp 'full_shard auto_wrap' --fsdp_transformer_layer_cls_to_wrap LlamaDecoderLayer --tf32 True --model_max_length 2048 --gradient_checkpointing True --num-gpus 1


deepspeed fastchat/train/train_lora.py
--model_name_or_path lmsys/vicuna-7b-v1.5
--lora_r 2
--lora_alpha 16
--lora_dropout 0.05
--data_path /home/n/FastChat/data/dummy_conversation.json
--bf16 True
--output_dir ./checkpoints
--num_train_epochs 3
--per_device_train_batch_size 1
--per_device_eval_batch_size 1
--gradient_accumulation_steps 1
--evaluation_strategy "no"
--save_strategy "steps"
--save_steps 1200
--save_total_limit 100
--learning_rate 2e-5
--weight_decay 0.
--warmup_ratio 0.03
--lr_scheduler_type "cosine"
--logging_steps 1
--tf32 True
--model_max_length 2048
--q_lora True
--deepspeed /home/n/FastChat/playground/deepspeed_config_s2.json`



torchrun --nproc_per_node=4 --master_port=20001 fastchat/train/train_mem.py \
    --model_name_or_path meta-llama/Llama-2-7b-hf \
    --data_path data/dummy_conversation.json \
    --bf16 True \
    --output_dir output_vicuna \
    --num_train_epochs 3 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --gradient_accumulation_steps 16 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 1200 \
    --save_total_limit 10 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --fsdp "full_shard auto_wrap" \
    --fsdp_transformer_layer_cls_to_wrap 'LlamaDecoderLayer' \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --lazy_preprocess True





# !torchrun --nproc_per_node=8 --master_port=20031 fastchat/train/train_mem.py \
#     --model_name_or_path /path/hf_model/llama-7b  \
#     --data_path /path/to/prompted_agent_dialogue_$CHARACTER.json \
#     --already_preprocess True \
#     --bf16 True \
#     --output_dir /path/to/ckpt/${CHARACTER}_7b \
#     --num_train_epochs 10 \
#     --per_device_train_batch_size 2 \
#     --per_device_eval_batch_size 16 \
#     --gradient_accumulation_steps 4 \
#     --evaluation_strategy epoch \
#     --save_strategy epoch \
#     --save_total_limit 10 \
#     --learning_rate 2e-5 \
#     --weight_decay 0.1 \
#     --warmup_ratio 0.04 \
#     --lr_scheduler_type cosine \
#     --logging_steps 1 \
#     --fsdp 'full_shard auto_wrap' \
#     --fsdp_transformer_layer_cls_to_wrap LlamaDecoderLayer \
#     --tf32 True \
#     --model_max_length 2048 \
#     --gradient_checkpointing True 


In [ ]:
# # one WSL
# # conda create -n vllmenv python=3.9
# # conda activate vllmenv
# # pip install transformers
# # pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
# # pip install git+https://github.com/casper-hansen/AutoAWQ.git
# # pip install vllm
# # pip install -U ipywidgets

# conda activate vllmenv
# python3 -m vllm.entrypoints.api_server --model TheBloke/Llama-2-7b-Chat-AWQ --quantization awq


# # python -m vllm.entrypoints.openai.api_server --model TheBloke/Llama-2-7B-Chat-AWQ --api-key token-abc123

In [ ]:
# from openai import OpenAI
# client = OpenAI(
#     base_url="http://localhost:8000/v1",
#     api_key="token-abc123",
# )

# completion = client.chat.completions.create(
#   model="mistralai/Mistral-7B-Instruct-v0.1",
#   messages=[
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "Hello!"}
#   ]
# )

# print(completion.choices[0].message)